# Procesamiento de Elecciones de Ayuntamiento - Manzanillo (2012-2024)
## Objetivo
El propósito de este notebook es limpiar, estandarizar y estructurar los resultados electorales para las elecciones de Ayuntamiento en Manzanillo, abarcando el periodo de 2012 a 2024. El objetivo final es generar archivos de datos limpios y consistentes que puedan ser fácilmente consolidados en una única base de datos para su posterior análisis y visualización geoespacial.

### Manzanillo AYUNTAMIENTO 2012

In [1]:
import pandas as pd 

df = pd.read_csv("/Users/omartellez/Manzanillo/01_datos_brutos/Ayuntamientos/Manzanillo_Ayuntamientos_2012.csv")

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   seccion_casilla  198 non-null    object
 1   pan              198 non-null    int64 
 2   pri_na           198 non-null    int64 
 3   prd              198 non-null    int64 
 4   pt               198 non-null    int64 
 5   pvem             198 non-null    int64 
 6   mc               198 non-null    int64 
 7   adc              198 non-null    int64 
 8   voto_nulo        198 non-null    int64 
 9   voto_total       198 non-null    int64 
dtypes: int64(9), object(1)
memory usage: 15.6+ KB


In [3]:
df.head()

,seccion_casilla,pan,pri_na,prd,pt,pvem,mc,adc,voto_nulo,voto_total
0,200 B1,167,195,12,5,5,18,0,15,417
1,201 B1,49,37,3,3,2,2,0,1,97
2,202 B1,167,146,12,7,8,2,0,26,368
3,202 C1,137,158,15,10,11,4,0,40,375
4,202 C2,158,138,13,2,10,2,0,24,347


In [4]:
import pandas as pd
import numpy as np
import os

# --- 1. CONFIGURACIÓN DE RUTAS ---
BASE_DIR = '/Users/omartellez/Manzanillo' 

input_folder = os.path.join(BASE_DIR, '01_datos_brutos', 'Ayuntamientos')
output_folder = os.path.join(BASE_DIR, '02_datos_limpios', 'Ayuntamientos')

input_file = 'Manzanillo_Ayuntamientos_2012.csv'
output_file = 'Manzanillo_Ayuntamientos_2012_limpio.csv'

# --- 2. CARGA DE DATOS ---
file_path = os.path.join(input_folder, input_file)
df = pd.read_csv(file_path)

# --- 3. LIMPIEZA Y ESTRUCTURACIÓN ---
df_mapped = df.copy()

# a) Separar 'seccion' y 'casilla' (necesario para este archivo)
df_mapped[['seccion', 'casilla']] = df_mapped['seccion_casilla'].str.extract(r'(\d+)\s*(.*)', expand=True)
df_mapped['seccion'] = pd.to_numeric(df_mapped['seccion'], errors='coerce')
df_mapped = df_mapped.drop(columns=['seccion_casilla'])

# b) Estandarizar nombres de partidos
df_mapped = df_mapped.rename(columns={'pri_na': 'pri_panal'})

# c) Asegurar que las columnas estándar existan
columnas_estandar = ['lista_nominal', 'voto_no_registrado']
for col in columnas_estandar:
    if col not in df_mapped.columns:
        df_mapped[col] = 0

# --- 4. CÁLCULO DE MÉTRICAS ANALÍTICAS (MODELO COMPLETO) ---

# a) Definir competidores y bloques para 2012
competidores_2012 = ['pan', 'pri_panal', 'prd', 'pt', 'pvem', 'mc', 'adc']
proxy_morena_2012 = ['prd', 'pt', 'mc']
oposicion_2012 = ['pan', 'pri_panal', 'pvem', 'adc']

# b) Cálculo de Bloques (Análisis Macro)
df_mapped['morena_coalitions'] = df_mapped[proxy_morena_2012].sum(axis=1)
df_mapped['non_morena_coalitions'] = df_mapped[oposicion_2012].sum(axis=1)

# c) Composición del Voto "Morena" (Análisis Fino)
df_mapped['voto_morena_solo'] = 0 # No existía el partido
df_mapped['voto_morena_aliados'] = df_mapped[proxy_morena_2012].sum(axis=1)

# d) Ganador por Casilla (Análisis Micro)
df_competidores = df_mapped[competidores_2012]
df_mapped['ganador_casilla_partido'] = df_competidores.idxmax(axis=1)
df_mapped['ganador_casilla_votos'] = df_competidores.max(axis=1)

# e) Cálculo del Segundo Lugar y Margen (Análisis de Competitividad)
# Usamos numpy para obtener los valores del 2do lugar de forma eficiente
segundo_lugar_votos = np.sort(df_competidores.values, axis=1)[:, -2]
df_mapped['segundo_lugar_votos'] = segundo_lugar_votos
# Para encontrar el nombre del 2do lugar, es un poco más complejo
# Creamos un df temporal donde el valor del ganador es nulo para que el nuevo max sea el 2do lugar
temp_df = df_competidores.copy()
for index, row in temp_df.iterrows():
    winner_col = df_mapped.loc[index, 'ganador_casilla_partido']
    temp_df.loc[index, winner_col] = np.nan
df_mapped['segundo_lugar_partido'] = temp_df.idxmax(axis=1)

df_mapped['margen_victoria'] = df_mapped['ganador_casilla_votos'] - df_mapped['segundo_lugar_votos']

# --- 5. ORGANIZACIÓN FINAL Y GUARDADO ---
columnas_finales = [
    'seccion', 'casilla', 'lista_nominal',
    # Columnas de votos originales
    'pan', 'pri_panal', 'prd', 'pt', 'pvem', 'mc', 'adc',
    # Grupo 1: Bloques Políticos (Macro)
    'morena_coalitions', 'non_morena_coalitions',
    # Grupo 2: Composición del Voto Morena (Fino)
    'voto_morena_solo', 'voto_morena_aliados',
    # Grupo 3: Competencia por Casilla (Micro)
    'ganador_casilla_partido', 'ganador_casilla_votos',
    'segundo_lugar_partido', 'segundo_lugar_votos', 'margen_victoria',
    # Votos adicionales
    'voto_no_registrado', 'voto_nulo', 'voto_total'
]
df_mapped = df_mapped[columnas_finales]

print("\nDatos limpios de 2012 con el modelo de análisis completo:")
df_mapped.info()
print(df_mapped[['seccion', 'ganador_casilla_partido', 'ganador_casilla_votos', 'segundo_lugar_partido', 'margen_victoria']].head())

# Guardar el archivo limpio
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, output_file)
df_mapped.to_csv(output_path, index=False)

print(f"\nArchivo limpio y actualizado de 2012 guardado en: {output_path}")


Datos limpios de 2012 con el modelo de análisis completo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   seccion                  198 non-null    int64 
 1   casilla                  198 non-null    object
 2   lista_nominal            198 non-null    int64 
 3   pan                      198 non-null    int64 
 4   pri_panal                198 non-null    int64 
 5   prd                      198 non-null    int64 
 6   pt                       198 non-null    int64 
 7   pvem                     198 non-null    int64 
 8   mc                       198 non-null    int64 
 9   adc                      198 non-null    int64 
 10  morena_coalitions        198 non-null    int64 
 11  non_morena_coalitions    198 non-null    int64 
 12  voto_morena_solo         198 non-null    int64 
 13  voto_morena_aliados      198 non-nul

### Manzanillo Ayuntamientos 2015

In [5]:
input_file = "Manzanillo_Ayuntamientos_2015.csv"
output_file = 'Manzanillo_Ayuntamientos_2015_limpio.csv'

file_path = os.path.join(input_folder, input_file)
df = pd.read_csv(file_path)

print("Datos originales:")
print(df.info())

Datos originales:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   seccion             216 non-null    int64 
 1   tipo_casilla        216 non-null    object
 2   pan                 216 non-null    int64 
 3   pri                 216 non-null    int64 
 4   prd                 216 non-null    int64 
 5   pvem                216 non-null    int64 
 6   pt                  216 non-null    int64 
 7   mc                  216 non-null    int64 
 8   na                  216 non-null    int64 
 9   morena              216 non-null    int64 
 10  humanista           216 non-null    int64 
 11  es                  216 non-null    int64 
 12  pri_pvem_panal      216 non-null    int64 
 13  voto_no_registrado  216 non-null    int64 
 14  voto_nulo           216 non-null    int64 
 15  voto_total          216 non-null    int64 
 16  lista_no

In [6]:
df.head(5)

,seccion,tipo_casilla,pan,pri,prd,pvem,pt,mc,na,morena,humanista,es,pri_pvem_panal,voto_no_registrado,voto_nulo,voto_total,lista_nominal
0,200,B1,191,84,4,34,3,11,1,4,10,0,3,0,4,349,619
1,201,B1,41,28,1,2,1,2,1,2,2,0,2,0,1,83,129
2,202,B1,190,87,3,14,4,6,3,4,4,6,5,0,11,337,620
3,202,C1,185,98,1,17,6,5,3,6,4,1,11,0,7,344,620
4,202,C2,171,122,3,15,5,8,3,5,1,2,10,0,3,348,620


### Manzanillo Ayuntamientos 2015

In [7]:
# --- 1. CONFIGURACIÓN DE RUTAS ---
BASE_DIR = '/Users/omartellez/Manzanillo' 

input_folder = os.path.join(BASE_DIR, '01_datos_brutos', 'Ayuntamientos')
output_folder = os.path.join(BASE_DIR, '02_datos_limpios', 'Ayuntamientos')

# CAMBIO: Nombres de archivo para 2015
input_file = 'Manzanillo_Ayuntamientos_2015.csv'
output_file = 'Manzanillo_Ayuntamientos_2015_limpio.csv'

# --- 2. CARGA DE DATOS ---
file_path = os.path.join(input_folder, input_file)
df = pd.read_csv(file_path)

# --- 3. LIMPIEZA Y ESTRUCTURACIÓN ---
df_mapped = df.copy()

# CAMBIO: Limpieza mucho más simple para este archivo
df_mapped = df_mapped.rename(columns={'tipo_casilla': 'casilla'})

# NO SE NECESITA: Separar seccion_casilla.
# NO SE NECESITA: Crear columnas estándar como lista_nominal.

# --- 4. CÁLCULO DE MÉTRICAS ANALÍTICAS (MODELO COMPLETO) ---

# a) Definir competidores y bloques para 2015
competidores_2015 = [
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'na', 
    'morena', 'humanista', 'es', 'pri_pvem_panal'
]
partidos_morena_2015 = ['morena']
oposicion_2015 = [
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'na', 
    'humanista', 'es', 'pri_pvem_panal'
]

# b) Cálculo de Bloques (Análisis Macro)
df_mapped['morena_coalitions'] = df_mapped[partidos_morena_2015].sum(axis=1)
df_mapped['non_morena_coalitions'] = df_mapped[oposicion_2015].sum(axis=1)

# c) Composición del Voto "Morena" (Análisis Fino)
df_mapped['voto_morena_solo'] = df_mapped['morena'] # Morena ya existe como partido
df_mapped['voto_morena_aliados'] = 0 # Compitió en solitario en esta elección

# d) Ganador por Casilla (Análisis Micro)
df_competidores = df_mapped[competidores_2015]
df_mapped['ganador_casilla_partido'] = df_competidores.idxmax(axis=1)
df_mapped['ganador_casilla_votos'] = df_competidores.max(axis=1)

# e) Cálculo del Segundo Lugar y Margen (Análisis de Competitividad)
segundo_lugar_votos = np.sort(df_competidores.values, axis=1)[:, -2]
df_mapped['segundo_lugar_votos'] = segundo_lugar_votos

temp_df = df_competidores.copy()
# Bucle para invalidar al ganador y encontrar al segundo lugar
for index, row in temp_df.iterrows():
    winner_col = df_mapped.loc[index, 'ganador_casilla_partido']
    # Asegurarse de que winner_col es un string válido antes de usarlo
    if pd.notna(winner_col) and winner_col in temp_df.columns:
        temp_df.loc[index, winner_col] = np.nan
df_mapped['segundo_lugar_partido'] = temp_df.idxmax(axis=1)

df_mapped['margen_victoria'] = df_mapped['ganador_casilla_votos'] - df_mapped['segundo_lugar_votos']

# --- 5. ORGANIZACIÓN FINAL Y GUARDADO ---
columnas_finales = [
    'seccion', 'casilla', 'lista_nominal',
    # Columnas de votos originales
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'na', 'morena', 'humanista', 'es', 'pri_pvem_panal',
    # Grupo 1: Bloques Políticos (Macro)
    'morena_coalitions', 'non_morena_coalitions',
    # Grupo 2: Composición del Voto Morena (Fino)
    'voto_morena_solo', 'voto_morena_aliados',
    # Grupo 3: Competencia por Casilla (Micro)
    'ganador_casilla_partido', 'ganador_casilla_votos',
    'segundo_lugar_partido', 'segundo_lugar_votos', 'margen_victoria',
    # Votos adicionales
    'voto_no_registrado', 'voto_nulo', 'voto_total'
]
df_mapped = df_mapped[columnas_finales]

print("\nDatos limpios de 2015 con el modelo de análisis completo:")
df_mapped.info()
print(df_mapped[['seccion', 'ganador_casilla_partido', 'ganador_casilla_votos', 'segundo_lugar_partido', 'margen_victoria']].head())

# Guardar el archivo limpio
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, output_file)
df_mapped.to_csv(output_path, index=False)

print(f"\nArchivo limpio y actualizado de 2015 guardado en: {output_path}")


Datos limpios de 2015 con el modelo de análisis completo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   seccion                  216 non-null    int64 
 1   casilla                  216 non-null    object
 2   lista_nominal            216 non-null    int64 
 3   pan                      216 non-null    int64 
 4   pri                      216 non-null    int64 
 5   prd                      216 non-null    int64 
 6   pvem                     216 non-null    int64 
 7   pt                       216 non-null    int64 
 8   mc                       216 non-null    int64 
 9   na                       216 non-null    int64 
 10  morena                   216 non-null    int64 
 11  humanista                216 non-null    int64 
 12  es                       216 non-null    int64 
 13  pri_pvem_panal           216 non-nul

### Manzanillo Ayuntamientos 2018

In [8]:
input_file = "Manzanillo_Ayuntamientos_2018.csv"
output_file = 'Manzanillo_Ayuntamientos_2018_limpio.csv'

file_path = os.path.join(input_folder, input_file)
df = pd.read_csv(file_path)

print("Datos originales:")
print(df.info())
df.head(5)

Datos originales:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233 entries, 0 to 232
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   seccion             233 non-null    int64 
 1   tipo_casilla        233 non-null    object
 2   pan                 233 non-null    int64 
 3   pri                 233 non-null    int64 
 4   prd                 233 non-null    int64 
 5   pt                  233 non-null    int64 
 6   pvem                233 non-null    int64 
 7   morena              233 non-null    int64 
 8   es                  233 non-null    int64 
 9   pan_prd             233 non-null    int64 
 10  pri_pvem            233 non-null    int64 
 11  morena_pt_es        233 non-null    int64 
 12  morena_pt           233 non-null    int64 
 13  pt_es               233 non-null    int64 
 14  morena_es           233 non-null    int64 
 15  mc                  233 non-null    int64 
 16  na      

,seccion,tipo_casilla,pan,pri,prd,pt,pvem,morena,es,pan_prd,...,morena_pt_es,morena_pt,pt_es,morena_es,mc,na,Independiente 6,voto_no_registrado,voto_nulo,voto_total
0,200,B,67,26,0,6,126,95,2,0,...,3,0,0,2,7,6,24,0,9,381
1,201,B,22,5,1,1,15,17,1,1,...,0,0,0,0,6,1,3,0,3,77
2,202,B,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,202,C,39,36,2,10,80,119,8,1,...,1,2,0,0,13,8,41,0,19,381
4,202,C,45,58,6,7,62,114,2,0,...,4,0,0,0,16,16,44,0,0,377


In [9]:
# --- 3. LIMPIEZA Y ESTRUCTURACIÓN ---
df_mapped = df.copy()

# a) Estandarizar nombres de columnas
df_mapped = df_mapped.rename(columns={
    'tipo_casilla': 'casilla',
    'Independiente 6': 'independiente_6'
})

# b) Asegurar que 'lista_nominal' exista
if 'lista_nominal' not in df_mapped.columns:
    df_mapped['lista_nominal'] = 0

# --- 4. CÁLCULO DE MÉTRICAS ANALÍTICAS (MODELO COMPLETO) ---

# a) Definir competidores y bloques para 2018
competidores_2018 = [
    'pan', 'pri', 'prd', 'pt', 'pvem', 'morena', 'es', 'mc', 'na',
    'pan_prd', 'pri_pvem',
    'morena_pt_es', 'morena_pt', 'pt_es', 'morena_es',
    'independiente_6'
]
partidos_morena_2018_total = [
    'morena', 'pt', 'es', 'morena_pt_es', 'morena_pt', 'pt_es', 'morena_es'
]
partidos_morena_2018_aliados = [
    'pt', 'es', 'morena_pt_es', 'morena_pt', 'pt_es', 'morena_es'
]
oposicion_2018 = [
    'pan', 'pri', 'prd', 'pvem', 'mc', 'na',
    'pan_prd', 'pri_pvem', 'independiente_6'
]

# b) Cálculo de Bloques (Análisis Macro)
df_mapped['morena_coalitions'] = df_mapped[partidos_morena_2018_total].sum(axis=1)
df_mapped['non_morena_coalitions'] = df_mapped[oposicion_2018].sum(axis=1)

# c) Composición del Voto "Morena" (Análisis Fino)
df_mapped['voto_morena_solo'] = df_mapped['morena']
df_mapped['voto_morena_aliados'] = df_mapped[partidos_morena_2018_aliados].sum(axis=1)

# d) Ganador por Casilla (Análisis Micro)
df_competidores = df_mapped[competidores_2018]
df_mapped['ganador_casilla_partido'] = df_competidores.idxmax(axis=1)
df_mapped['ganador_casilla_votos'] = df_competidores.max(axis=1)

# e) Cálculo del Segundo Lugar y Margen (Análisis de Competitividad)
segundo_lugar_votos = np.sort(df_competidores.values, axis=1)[:, -2]
df_mapped['segundo_lugar_votos'] = segundo_lugar_votos

temp_df = df_competidores.copy()
for index, row in temp_df.iterrows():
    winner_col = df_mapped.loc[index, 'ganador_casilla_partido']
    if pd.notna(winner_col) and winner_col in temp_df.columns:
        temp_df.loc[index, winner_col] = np.nan
df_mapped['segundo_lugar_partido'] = temp_df.idxmax(axis=1)

df_mapped['margen_victoria'] = df_mapped['ganador_casilla_votos'] - df_mapped['segundo_lugar_votos']

# --- 5. ORGANIZACIÓN FINAL Y GUARDADO ---
columnas_finales = [
    'seccion', 'casilla', 'lista_nominal',
    # Columnas de votos originales
    'pan', 'pri', 'prd', 'pt', 'pvem', 'morena', 'es', 'mc', 'na', 'pan_prd', 
    'pri_pvem', 'morena_pt_es', 'morena_pt', 'pt_es', 'morena_es', 'independiente_6',
    # Grupo 1: Bloques Políticos (Macro)
    'morena_coalitions', 'non_morena_coalitions',
    # Grupo 2: Composición del Voto Morena (Fino)
    'voto_morena_solo', 'voto_morena_aliados',
    # Grupo 3: Competencia por Casilla (Micro)
    'ganador_casilla_partido', 'ganador_casilla_votos',
    'segundo_lugar_partido', 'segundo_lugar_votos', 'margen_victoria',
    # Votos adicionales
    'voto_no_registrado', 'voto_nulo', 'voto_total'
]
df_mapped = df_mapped[columnas_finales]

print("\nDatos limpios de 2018 con el modelo de análisis completo:")
df_mapped.info()
print(df_mapped[['seccion', 'ganador_casilla_partido', 'ganador_casilla_votos', 'segundo_lugar_partido', 'margen_victoria']].head())

# Guardar el archivo limpio
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, output_file)
df_mapped.to_csv(output_path, index=False)

print(f"\nArchivo limpio y actualizado de 2018 guardado en: {output_path}")


Datos limpios de 2018 con el modelo de análisis completo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233 entries, 0 to 232
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   seccion                  233 non-null    int64 
 1   casilla                  233 non-null    object
 2   lista_nominal            233 non-null    int64 
 3   pan                      233 non-null    int64 
 4   pri                      233 non-null    int64 
 5   prd                      233 non-null    int64 
 6   pt                       233 non-null    int64 
 7   pvem                     233 non-null    int64 
 8   morena                   233 non-null    int64 
 9   es                       233 non-null    int64 
 10  mc                       233 non-null    int64 
 11  na                       233 non-null    int64 
 12  pan_prd                  233 non-null    int64 
 13  pri_pvem                 233 non-nul

### Manzanillo Ayuntamientos 2021

In [10]:
input_file = "Manzanillo_Ayuntamientos_2021.csv"
output_file = 'Manzanillo_Ayuntamientos_2021_limpio.csv'

file_path = os.path.join(input_folder, input_file)
df = pd.read_csv(file_path)

print("Datos originales:")
print(df.info())
df.head(5)

Datos originales:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239 entries, 0 to 238
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CIRCUNSCRIPCION           0 non-null      float64
 1   ID_ESTADO                 239 non-null    int64  
 2   NOMBRE_ESTADO             239 non-null    object 
 3   ID_DISTRITO_LOCAL         239 non-null    int64  
 4   CABECERA_DISTRITAL_LOCAL  239 non-null    object 
 5   ID_MUNICIPIO              239 non-null    int64  
 6   municipio                 239 non-null    object 
 7   seccion                   239 non-null    int64  
 8   casilla                   239 non-null    object 
 9   EXT_CONTIGUA              239 non-null    int64  
 10  pan                       239 non-null    int64  
 11  pri                       239 non-null    int64  
 12  prd                       239 non-null    int64  
 13  pvem                      239 non-null    int64

,CIRCUNSCRIPCION,ID_ESTADO,NOMBRE_ESTADO,ID_DISTRITO_LOCAL,CABECERA_DISTRITAL_LOCAL,ID_MUNICIPIO,municipio,seccion,casilla,EXT_CONTIGUA,...,pri_prd,NUM_VOTOS_VALIDOS,voto_no_registrado,voto_nulo,voto_total,lista_nominal,ESTATUS_ACTA,TRIBUNAL,OBSERVACIONES,RUTA_ACTA
0,NaN,6,COLIMA,13,MANZANILLO,8,MANZANILLO,200,B,0,...,0,347,0,5,352,709,GRUPO DE RECUENTO,NaN,NaN,NaN
1,NaN,6,COLIMA,13,MANZANILLO,8,MANZANILLO,202,B,0,...,0,280,1,10,291,605,GRUPO DE RECUENTO,NaN,NaN,NaN
2,NaN,6,COLIMA,13,MANZANILLO,8,MANZANILLO,202,C,0,...,0,299,0,5,304,605,GRUPO DE RECUENTO,NaN,NaN,NaN
3,NaN,6,COLIMA,13,MANZANILLO,8,MANZANILLO,202,C,0,...,0,291,0,10,301,604,ACTA CASILLA,NaN,NaN,NaN
4,NaN,6,COLIMA,13,MANZANILLO,8,MANZANILLO,202,C,0,...,0,297,0,5,302,604,GRUPO DE RECUENTO,NaN,NaN,NaN


In [11]:

# --- 3. LIMPIEZA Y ESTRUCTURACIÓN ---
df_mapped = df.copy()

# a) Estandarizar todos los nombres de columnas a minúsculas
df_mapped.columns = df_mapped.columns.str.lower()

# b) Eliminar columnas innecesarias (de contexto, vacías, redundantes)
columnas_a_eliminar = [
    'circunscripcion', 'id_estado', 'nombre_estado', 'id_distrito_local', 
    'cabecera_distrital_local', 'id_municipio', 'municipio', 'id_casilla', 
    'ext_contigua', 'casilla.1', 'casilla', # Se eliminan ambas y se usa la original 'casilla' del df
    'num_votos_validos', 'estatus_acta', 'tribunal', 'observaciones', 'ruta_acta'
]
# Renombramos la columna 'casilla' original para evitar conflicto antes de la limpieza
df_mapped = df_mapped.rename(columns={'casilla': 'casilla_id'})
df.columns = df.columns.str.lower() # Renombrar también las del df original
columnas_a_eliminar = [col for col in columnas_a_eliminar if col in df_mapped.columns]
df_mapped = df_mapped.drop(columns=columnas_a_eliminar, errors='ignore').rename(columns={'casilla_id':'casilla'})


# NO SE NECESITA: Crear 'lista_nominal', ya existe.

# --- 4. CÁLCULO DE MÉTRICAS ANALÍTICAS (MODELO COMPLETO) ---

# a) Definir competidores y bloques para 2021
competidores_2021 = [
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'morena', 'na', 'es', 'rsp', 'fxm',
    'morena_na', 'pan_pri_prd', 'pan_pri', 'pan_prd', 'pri_prd'
]
partidos_morena_2021_total = ['morena', 'na', 'morena_na']
partidos_morena_2021_aliados = ['na', 'morena_na']
oposicion_2021 = [
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'es', 'rsp', 'fxm',
    'pan_pri_prd', 'pan_pri', 'pan_prd', 'pri_prd'
]

# b) Cálculo de Bloques (Análisis Macro)
df_mapped['morena_coalitions'] = df_mapped[partidos_morena_2021_total].sum(axis=1)
df_mapped['non_morena_coalitions'] = df_mapped[oposicion_2021].sum(axis=1)

# c) Composición del Voto "Morena" (Análisis Fino)
df_mapped['voto_morena_solo'] = df_mapped['morena']
df_mapped['voto_morena_aliados'] = df_mapped[partidos_morena_2021_aliados].sum(axis=1)

# d) Ganador por Casilla (Análisis Micro)
df_competidores = df_mapped[competidores_2021]
df_mapped['ganador_casilla_partido'] = df_competidores.idxmax(axis=1)
df_mapped['ganador_casilla_votos'] = df_competidores.max(axis=1)

# e) Cálculo del Segundo Lugar y Margen (Análisis de Competitividad)
segundo_lugar_votos = np.sort(df_competidores.values, axis=1)[:, -2]
df_mapped['segundo_lugar_votos'] = segundo_lugar_votos

temp_df = df_competidores.copy()
for index, row in temp_df.iterrows():
    winner_col = df_mapped.loc[index, 'ganador_casilla_partido']
    if pd.notna(winner_col) and winner_col in temp_df.columns:
        temp_df.loc[index, winner_col] = np.nan
df_mapped['segundo_lugar_partido'] = temp_df.idxmax(axis=1)

df_mapped['margen_victoria'] = df_mapped['ganador_casilla_votos'] - df_mapped['segundo_lugar_votos']

# --- 5. ORGANIZACIÓN FINAL Y GUARDADO ---
columnas_finales = [
    'seccion', 'casilla', 'lista_nominal',
    # Columnas de votos originales
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'morena', 'na', 'es', 'rsp', 'fxm',
    'morena_na', 'pan_pri_prd', 'pan_pri', 'pan_prd', 'pri_prd',
    # Grupo 1: Bloques Políticos (Macro)
    'morena_coalitions', 'non_morena_coalitions',
    # Grupo 2: Composición del Voto Morena (Fino)
    'voto_morena_solo', 'voto_morena_aliados',
    # Grupo 3: Competencia por Casilla (Micro)
    'ganador_casilla_partido', 'ganador_casilla_votos',
    'segundo_lugar_partido', 'segundo_lugar_votos', 'margen_victoria',
    # Votos adicionales
    'voto_no_registrado', 'voto_nulo', 'voto_total'
]
df_mapped = df_mapped[columnas_finales]

print("\nDatos limpios de 2021 con el modelo de análisis completo:")
df_mapped.info()
print(df_mapped[['seccion', 'ganador_casilla_partido', 'ganador_casilla_votos', 'segundo_lugar_partido', 'margen_victoria']].head())

# Guardar el archivo limpio
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, output_file)
df_mapped.to_csv(output_path, index=False)

print(f"\nArchivo limpio y actualizado de 2021 guardado en: {output_path}")


Datos limpios de 2021 con el modelo de análisis completo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239 entries, 0 to 238
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   seccion                  239 non-null    int64 
 1   casilla                  239 non-null    object
 2   lista_nominal            239 non-null    int64 
 3   pan                      239 non-null    int64 
 4   pri                      239 non-null    int64 
 5   prd                      239 non-null    int64 
 6   pvem                     239 non-null    int64 
 7   pt                       239 non-null    int64 
 8   mc                       239 non-null    int64 
 9   morena                   239 non-null    int64 
 10  na                       239 non-null    int64 
 11  es                       239 non-null    int64 
 12  rsp                      239 non-null    int64 
 13  fxm                      239 non-nul

### Manzanillo Ayuntamientos 2024

In [12]:
input_file = "Manzanillo_Ayuntamientos_2024.csv"
output_file = 'Manzanillo_Ayuntamientos_2024_limpio.csv'

file_path = os.path.join(input_folder, input_file)
df = pd.read_csv(file_path)

print("Datos originales:")
print(df.info())
df.head(5)

Datos originales:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ID_ESTADO              256 non-null    int64 
 1   ID_MUNICIPIO_LOCAL     256 non-null    int64 
 2   alcaldia               256 non-null    object
 3   seccion                256 non-null    object
 4   lista_nominal          256 non-null    int64 
 5   pan                    256 non-null    int64 
 6   pri                    256 non-null    int64 
 7   prd                    256 non-null    int64 
 8   pvem                   256 non-null    int64 
 9   pt                     256 non-null    int64 
 10  mc                     256 non-null    int64 
 11  morena                 256 non-null    int64 
 12  na                     256 non-null    int64 
 13  es                     256 non-null    int64 
 14  fxm                    256 non-null    int64 
 15  pan_p

,ID_ESTADO,ID_MUNICIPIO_LOCAL,alcaldia,seccion,lista_nominal,pan,pri,prd,pvem,pt,...,pvem_pt_morena,pvem_morena,pvem_pt,pt_morena,voto_nulo,voto_no_registrado,voto_valido,voto_total,NUM_BOLETAS_RECIBIDAS,NUM_BOLETAS_SOBRANTES
0,6,8,Manzanillo,0000 VA 8 Ayun,4,2,0,0,0,1,...,0,0,0,0,0,0,4,4,4,0
1,6,8,Manzanillo,0200 B·sica 1,673,43,46,7,26,2,...,16,1,0,3,10,0,375,385,707,321
2,6,8,Manzanillo,0202 B·sica 1,591,21,29,3,16,8,...,14,2,0,0,14,0,298,312,625,299
3,6,8,Manzanillo,0202 Contigua 1,591,28,35,1,18,3,...,9,3,1,0,9,0,323,332,625,293
4,6,8,Manzanillo,0202 Contigua 2,591,26,44,7,17,8,...,5,0,0,1,8,0,316,324,625,301


In [13]:
# --- 3. LIMPIEZA Y ESTRUCTURACIÓN ---
df_mapped = df.copy()

# a) Estandarizar todos los nombres de columnas a minúsculas
df_mapped.columns = df_mapped.columns.str.lower()

# b) Eliminar columnas innecesarias de este archivo
columnas_a_eliminar = [
    'id_estado', 'id_municipio_local', 'alcaldia', 'voto_valido', 
    'num_boletas_recibidas', 'num_boletas_sobrantes'
]
df_mapped = df_mapped.drop(columns=columnas_a_eliminar, errors='ignore')

# c) Separar 'seccion' y 'casilla' (VERSIÓN DEFINITIVA Y CORREGIDA)
# 1. Extraemos los datos a columnas con nombres temporales y únicos
df_mapped[['seccion_temp_num', 'casilla_temp']] = df_mapped['seccion'].str.extract(r'(\d+)\s*(.*)', expand=True)
# 2. Eliminamos la columna de texto original 'seccion' que ya no necesitamos
df_mapped = df_mapped.drop(columns=['seccion'])
# 3. Renombramos las columnas temporales a sus nombres finales
df_mapped = df_mapped.rename(columns={
    'seccion_temp_num': 'seccion',
    'casilla_temp': 'casilla'
})
# 4. Convertimos la nueva columna 'seccion' a tipo numérico
df_mapped['seccion'] = pd.to_numeric(df_mapped['seccion'], errors='coerce')


# --- 4. CÁLCULO DE MÉTRICAS ANALÍTICAS (MODELO COMPLETO) ---
# (Esta sección no cambia y es correcta)
competidores_2024 = [
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'morena', 'na', 'es', 'fxm',
    'pan_pri', 'pvem_pt_morena', 'pvem_morena', 'pvem_pt', 'pt_morena'
]
partidos_morena_2024_total = ['morena', 'pvem', 'pt', 'pvem_pt_morena', 'pvem_morena', 'pvem_pt', 'pt_morena']
partidos_morena_2024_aliados = ['pvem', 'pt', 'pvem_pt_morena', 'pvem_morena', 'pvem_pt', 'pt_morena']
oposicion_2024 = ['pan', 'pri', 'prd', 'mc', 'na', 'es', 'fxm', 'pan_pri']

df_mapped['morena_coalitions'] = df_mapped[partidos_morena_2024_total].sum(axis=1)
df_mapped['non_morena_coalitions'] = df_mapped[oposicion_2024].sum(axis=1)
df_mapped['voto_morena_solo'] = df_mapped['morena']
df_mapped['voto_morena_aliados'] = df_mapped[partidos_morena_2024_aliados].sum(axis=1)

df_competidores = df_mapped[competidores_2024]
df_mapped['ganador_casilla_partido'] = df_competidores.idxmax(axis=1)
df_mapped['ganador_casilla_votos'] = df_competidores.max(axis=1)

segundo_lugar_votos = np.sort(df_competidores.values, axis=1)[:, -2]
df_mapped['segundo_lugar_votos'] = segundo_lugar_votos
temp_df = df_competidores.copy()
for index, row in temp_df.iterrows():
    winner_col = df_mapped.loc[index, 'ganador_casilla_partido']
    if pd.notna(winner_col) and winner_col in temp_df.columns:
        temp_df.loc[index, winner_col] = np.nan
df_mapped['segundo_lugar_partido'] = temp_df.idxmax(axis=1)
df_mapped['margen_victoria'] = df_mapped['ganador_casilla_votos'] - df_mapped['segundo_lugar_votos']


# --- 5. ORGANIZACIÓN FINAL Y GUARDADO ---
columnas_finales = [
    'seccion', 'casilla', 'lista_nominal',
    'pan', 'pri', 'prd', 'pvem', 'pt', 'mc', 'morena', 'na', 'es', 'fxm', 'pan_pri',
    'pvem_pt_morena', 'pvem_morena', 'pvem_pt', 'pt_morena',
    'morena_coalitions', 'non_morena_coalitions',
    'voto_morena_solo', 'voto_morena_aliados',
    'ganador_casilla_partido', 'ganador_casilla_votos',
    'segundo_lugar_partido', 'segundo_lugar_votos', 'margen_victoria',
    'voto_no_registrado', 'voto_nulo', 'voto_total'
]
df_mapped = df_mapped[columnas_finales]

print("\nDatos limpios de 2024 (versión final) con el modelo de análisis completo:")
df_mapped.info()
print(df_mapped[['seccion', 'casilla', 'ganador_casilla_partido', 'ganador_casilla_votos', 'margen_victoria']].head())

# Guardar el archivo limpio
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, output_file)
df_mapped.to_csv(output_path, index=False)

print(f"\nArchivo limpio y actualizado de 2024 guardado en: {output_path}")


Datos limpios de 2024 (versión final) con el modelo de análisis completo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   seccion                  256 non-null    int64 
 1   casilla                  256 non-null    object
 2   lista_nominal            256 non-null    int64 
 3   pan                      256 non-null    int64 
 4   pri                      256 non-null    int64 
 5   prd                      256 non-null    int64 
 6   pvem                     256 non-null    int64 
 7   pt                       256 non-null    int64 
 8   mc                       256 non-null    int64 
 9   morena                   256 non-null    int64 
 10  na                       256 non-null    int64 
 11  es                       256 non-null    int64 
 12  fxm                      256 non-null    int64 
 13  pan_pri             

## Base consolidada elecciones Ayuntamientos en Manzanillo 2012-2024

In [15]:
import pandas as pd
import os
import glob

# --- 1. CONFIGURACIÓN ---
BASE_DIR = '/Users/omartellez/Manzanillo'
# CORRECCIÓN: El nombre de tu carpeta de Ayuntamientos
input_folder = os.path.join(BASE_DIR, '02_datos_limpios', 'Ayuntamientos') 
output_folder = os.path.join(BASE_DIR, '03_base_consolidada')
output_file = 'ayuntamiento_consolidado_2012_2024.csv'

# --- 2. ENCONTRAR Y CARGAR TODOS LOS ARCHIVOS LIMPIOS ---
search_pattern = os.path.join(input_folder, '*_limpio.csv')
clean_files = sorted(glob.glob(search_pattern))

if not clean_files:
    print("No se encontraron archivos limpios para consolidar.")
else:
    all_dfs = []
    print(f"Se encontraron {len(clean_files)} archivos para consolidar:")
    for file_path in clean_files:
        print(f"  -> Cargando: {os.path.basename(file_path)}")
        
        try:
            year = int(os.path.basename(file_path).split('_')[-2])
        except (ValueError, IndexError):
            year = None

        df = pd.read_csv(file_path)
        df['anio'] = year
        
        # <-- LÍNEA AÑADIDA Y CLAVE ---
        df['tipo_eleccion'] = 'Ayuntamiento'
        
        all_dfs.append(df)

    # --- 3. CONSOLIDAR, ORDENAR Y GUARDAR ---
    consolidated_df = pd.concat(all_dfs, ignore_index=True)
    
    # Lógica de ordenamiento automático y robusto
    columnas_inicio = [
        'seccion', 'casilla', 'anio', 'tipo_eleccion', 'lista_nominal',
        'ganador_casilla_partido', 'ganador_casilla_votos', 'segundo_lugar_partido', 
        'segundo_lugar_votos', 'margen_victoria',
        'morena_coalitions', 'non_morena_coalitions', 'voto_morena_solo', 'voto_morena_aliados'
    ]
    columnas_fin = ['voto_no_registrado', 'voto_nulo', 'voto_total']
    
    columnas_inicio_existentes = [col for col in columnas_inicio if col in consolidated_df.columns]
    columnas_fin_existentes = [col for col in columnas_fin if col in consolidated_df.columns]
    
    columnas_votos_crudos = sorted([
        col for col in consolidated_df.columns 
        if col not in columnas_inicio_existentes and col not in columnas_fin_existentes
    ])
    
    orden_final = columnas_inicio_existentes + columnas_votos_crudos + columnas_fin_existentes
    
    consolidated_df_ordenado = consolidated_df[orden_final]
    consolidated_df_final = consolidated_df_ordenado.fillna(0)
        
    # Guardar la base de datos consolidada
    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, output_file)
    consolidated_df_final.to_csv(output_path, index=False)
    
    print(f"\n¡Base de datos de Ayuntamiento consolidada creada exitosamente!")
    print(f"Guardada en: {output_path}")
    print("\nResumen de la base consolidada final:")
    consolidated_df_final.info()

Se encontraron 5 archivos para consolidar:
  -> Cargando: Manzanillo_Ayuntamientos_2012_limpio.csv
  -> Cargando: Manzanillo_Ayuntamientos_2015_limpio.csv
  -> Cargando: Manzanillo_Ayuntamientos_2018_limpio.csv
  -> Cargando: Manzanillo_Ayuntamientos_2021_limpio.csv
  -> Cargando: Manzanillo_Ayuntamientos_2024_limpio.csv

¡Base de datos de Ayuntamiento consolidada creada exitosamente!
Guardada en: /Users/omartellez/Manzanillo/03_base_consolidada/ayuntamiento_consolidado_2012_2024.csv

Resumen de la base consolidada final:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1142 entries, 0 to 1141
Data columns (total 47 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   seccion                  1142 non-null   int64  
 1   casilla                  1142 non-null   object 
 2   anio                     1142 non-null   int64  
 3   tipo_eleccion            1142 non-null   object 
 4   lista_nominal            1142 non-nu